In [1]:
%load_ext autoreload
%autoreload 2
import lm_eval.evaluator
import llminference as L

# Load a pretrained model & evaluate

In [2]:
adapter = L.Adapter.from_pretrained("EleutherAI/pythia-70m")
display(lm_eval.evaluator.evaluate(adapter, lm_eval.tasks.get_task_dict(["lambada_openai"])))

No config specified, defaulting to: lambada_openai/default
Found cached dataset lambada_openai (/nethome/douglaso/.cache/huggingface/datasets/EleutherAI___lambada_openai/default/1.0.0/57baddecfa09d1790541ef07274c5666abfbe9d2ccd0cd46013cd557b0343095)


  0%|          | 0/1 [00:00<?, ?it/s]

Running loglikelihood requests


100%|██████████| 5151/5151 [01:00<00:00, 84.81it/s] 


bootstrapping for stddev: perplexity


100%|██████████| 100/100 [00:02<00:00, 34.73it/s]


{'results': {'lambada_openai': {'ppl': 130.96083072941352,
   'ppl_stderr': 5.519581850216156,
   'acc': 0.22724626431205122,
   'acc_stderr': 0.005838227385425755}},
 'versions': {'lambada_openai': 0}}

In [5]:
outcompare_data = L.outcompare.Dataset.load("../data/pythia-70m.json")
display(L.outcompare.evaluate(adapter.model, outcompare_data, batch_size=16, limit=64))

{'entropy_rmse': 0.0,
 'entropy_rmse_stderr': 0.0,
 'exact_match_length': 64.0,
 'exact_match_length_stderr': 0.0,
 'edit_distance_L16': 0.0,
 'edit_distance_L16_stderr': 0.0}

## Deliberately mess up the model & see what happens

In [6]:
adapter.model.gpt_neox.layers[4].attention.dense.weight.data.fill_(0)
display(lm_eval.evaluator.evaluate(adapter, lm_eval.tasks.get_task_dict(["lambada_openai"])))

No config specified, defaulting to: lambada_openai/default
Found cached dataset lambada_openai (/nethome/douglaso/.cache/huggingface/datasets/EleutherAI___lambada_openai/default/1.0.0/57baddecfa09d1790541ef07274c5666abfbe9d2ccd0cd46013cd557b0343095)


  0%|          | 0/1 [00:00<?, ?it/s]

Running loglikelihood requests


100%|██████████| 5151/5151 [00:59<00:00, 86.14it/s] 


bootstrapping for stddev: perplexity


100%|██████████| 100/100 [00:02<00:00, 37.33it/s]


{'results': {'lambada_openai': {'ppl': 407.38169350433435,
   'ppl_stderr': 16.256237186731294,
   'acc': 0.11410828643508636,
   'acc_stderr': 0.004429564034890585}},
 'versions': {'lambada_openai': 0}}

In [7]:
display(L.outcompare.evaluate(adapter.model, outcompare_data, batch_size=16, limit=64))

{'entropy_rmse': 0.5335424542427063,
 'entropy_rmse_stderr': 0.02504601702094078,
 'exact_match_length': 5.140625,
 'exact_match_length_stderr': 1.142146110534668,
 'edit_distance_L16': 10.40625,
 'edit_distance_L16_stderr': 0.6003961563110352}